In [1]:
# RNN which will (hopefully) generate a random chat between 2 people

In [4]:
# get and unzip dataset - only need to run once
dataset = 'projjal1/human-conversation-training-data'
kaggle.api.dataset_download_files(dataset, path=".", unzip=True)

Dataset URL: https://www.kaggle.com/datasets/projjal1/human-conversation-training-data


In [28]:
# for kaggle
import kaggle
# for work
import torch
from nltk.tokenize import word_tokenize
from collections import Counter

In [11]:
# reading the data line by line into list
lines = []
dataset_file = open('human_chat.txt','r',encoding='utf-8')
while True:
    line_wise_data = dataset_file.readline()
    if not line_wise_data:
        break
    lines.append(line_wise_data)
for line in lines:
    print(line,end="")

Human 1: Hi!
Human 2: What is your favorite holiday?
Human 1: one where I get to meet lots of different people.
Human 2: What was the most number of people you have ever met during a holiday?
Human 1: Hard to keep a count. Maybe 25.
Human 2: Which holiday was that?
Human 1: I think it was Australia
Human 2: Do you still talk to the people you met?
Human 1: Not really. The interactions are usually short-lived but it's fascinating to learn where people are coming from and what matters to them
Human 2: Yea, me too. I feel like God often puts strangers in front of you, and gives you an opportunity to connect with them in that moment in deeply meaningful ways. Do you ever feel like you know things about strangers without them telling you?
Human 1: what do you mean?
Human 2: I think it's like a 6th sense, often seen as "cold readings" to people, but can be remarkably accurate. I once sat next to a man in a coffee and I felt a pain in my back. I asked the stranger if he had a pain. It turns o

In [37]:
# building vocabulary in token form
# words = [line.lower().split() for line in lines] # using nltk tokens for better splitting (fine! vs fine)
words = [word_tokenize(line.lower()) for line in lines]
# print(words)
word_counts = Counter([ word for word_line in words for word in word_line ])
# print(word_counts)
vocabulary = {word:i for i,(word,_) in enumerate(word_counts.items(),1)} # start vocal indexing from 1 because cleaner
print(vocabulary)

{'human': 1, '1': 2, ':': 3, 'hi': 4, '!': 5, '2': 6, 'what': 7, 'is': 8, 'your': 9, 'favorite': 10, 'holiday': 11, '?': 12, 'one': 13, 'where': 14, 'i': 15, 'get': 16, 'to': 17, 'meet': 18, 'lots': 19, 'of': 20, 'different': 21, 'people': 22, '.': 23, 'was': 24, 'the': 25, 'most': 26, 'number': 27, 'you': 28, 'have': 29, 'ever': 30, 'met': 31, 'during': 32, 'a': 33, 'hard': 34, 'keep': 35, 'count': 36, 'maybe': 37, '25': 38, 'which': 39, 'that': 40, 'think': 41, 'it': 42, 'australia': 43, 'do': 44, 'still': 45, 'talk': 46, 'not': 47, 'really': 48, 'interactions': 49, 'are': 50, 'usually': 51, 'short-lived': 52, 'but': 53, "'s": 54, 'fascinating': 55, 'learn': 56, 'coming': 57, 'from': 58, 'and': 59, 'matters': 60, 'them': 61, 'yea': 62, ',': 63, 'me': 64, 'too': 65, 'feel': 66, 'like': 67, 'god': 68, 'often': 69, 'puts': 70, 'strangers': 71, 'in': 72, 'front': 73, 'gives': 74, 'an': 75, 'opportunity': 76, 'connect': 77, 'with': 78, 'moment': 79, 'deeply': 80, 'meaningful': 81, 'ways':

In [43]:
# encoding the sentences according to the vocabulary
encoded_lines = [[vocabulary[word] for word in curr_line] for curr_line in words]
print(encoded_lines)

[[1, 2, 3, 4, 5], [1, 6, 3, 7, 8, 9, 10, 11, 12], [1, 2, 3, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23], [1, 6, 3, 7, 24, 25, 26, 27, 20, 22, 28, 29, 30, 31, 32, 33, 11, 12], [1, 2, 3, 34, 17, 35, 33, 36, 23, 37, 38, 23], [1, 6, 3, 39, 11, 24, 40, 12], [1, 2, 3, 15, 41, 42, 24, 43], [1, 6, 3, 44, 28, 45, 46, 17, 25, 22, 28, 31, 12], [1, 2, 3, 47, 48, 23, 25, 49, 50, 51, 52, 53, 42, 54, 55, 17, 56, 14, 22, 50, 57, 58, 59, 7, 60, 17, 61], [1, 6, 3, 62, 63, 64, 65, 23, 15, 66, 67, 68, 69, 70, 71, 72, 73, 20, 28, 63, 59, 74, 28, 75, 76, 17, 77, 78, 61, 72, 40, 79, 72, 80, 81, 82, 23, 44, 28, 30, 66, 67, 28, 83, 84, 85, 71, 86, 61, 87, 28, 12], [1, 2, 3, 7, 44, 28, 88, 12], [1, 6, 3, 15, 41, 42, 54, 67, 33, 89, 90, 63, 69, 91, 92, 93, 94, 95, 96, 17, 22, 63, 53, 97, 98, 99, 100, 23, 15, 101, 102, 103, 17, 33, 104, 72, 33, 105, 59, 15, 106, 33, 107, 72, 108, 109, 23, 15, 110, 25, 111, 112, 113, 114, 33, 107, 23, 42, 115, 116, 40, 113, 117, 72, 25, 118, 119, 63, 59, 120, 113, 121, 33, 122, 12

In [47]:
# padding the sentences to make input size uniform
longest_line_length = max([len(line) for line in encoded_lines]) # can replace encoded_lines with words as well
# print(longest_line_length)
padded_lines = [line + [0]*(longest_line_length - len(line)) for line in encoded_lines]
for line in padded_lines:
    print(line)

[1, 2, 3, 4, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 6, 3, 7, 8, 9, 10, 11, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,